#### Installations

In [66]:
!pip install nltk
!pip install bert_score
!pip install datasets
!pip install -U  torchdata
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

/home/mehar21541/anaconda3/envs/vis/lib/python3.11/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 34.2 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
/home/mehar21541/anaconda3/envs/vis/lib/python3.11/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and ins

#### Importing Libraries

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset
import tqdm
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from typing import Iterable, List
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
from timeit import default_timer as timer
import nltk 
from nltk.tokenize import word_tokenize
import evaluate
import bert_score
import wandb

In [5]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#### Loading data and data visualization

In [6]:
train_data = load_dataset("wmt16", "de-en", split="train[:50000]")
val_data = load_dataset("wmt16", "de-en", split="validation")
test_data = load_dataset("wmt16", "de-en", split="test")

In [7]:
print(f"Train data size: {len(train_data)}, type: {type(train_data)}")
print(f"Validation data size: {len(val_data)}, type: {type(val_data)}")
print(f"Test data size: {len(test_data)}, type: {type(test_data)}")

Train data size: 50000, type: <class 'datasets.arrow_dataset.Dataset'>
Validation data size: 2169, type: <class 'datasets.arrow_dataset.Dataset'>
Test data size: 2999, type: <class 'datasets.arrow_dataset.Dataset'>


In [8]:
for i in range(5):
    data = train_data[i]
    german = data["translation"]["de"]
    english = data["translation"]["en"]
    print(f"German: {german}")
    print(f"English: {english}")

German: Wiederaufnahme der Sitzungsperiode
English: Resumption of the session
German: Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie schöne Ferien hatten.
English: I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.
German: Wie Sie feststellen konnten, ist der gefürchtete "Millenium-Bug " nicht eingetreten. Doch sind Bürger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden.
English: Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.
German: Im Parlament besteht der Wunsch nach einer Aussprache im Verlauf dieser Sit

In [11]:
config = dict(
    source_language = "de", 
    target_language = "en",
    EMB_SIZE = 512,
    NHEAD = 8,
    FFN_HID_DIM = 512,
    BATCH_SIZE = 32,
    NUM_ENCODER_LAYERS = 3,
    NUM_DECODER_LAYERS = 3,
    NUM_EPOCHS = 5
)

In [12]:
source_language = config["source_language"]
target_language = config["target_language"]

#### Initialising tokenizer

In [13]:
# Tokenization
token_transform = {}
token_transform[source_language] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[target_language] = get_tokenizer('spacy', language='en_core_web_sm')

In [14]:
for i in range(5): 
    print(train_data[i])
    data_pt = train_data[i]
    data_src = data_pt['translation'][source_language]
    data_tgt = data_pt['translation'][target_language]
    print(f"German: {data_src}")
    print(f"English: {data_tgt}")
    print(f"Tokenized German: {token_transform[source_language](data_src)}")
    print(f"Tokenized English: {token_transform[target_language](data_tgt)}")
    break
    

{'translation': {'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption of the session'}}
German: Wiederaufnahme der Sitzungsperiode
English: Resumption of the session
Tokenized German: ['Wiederaufnahme', 'der', 'Sitzungsperiode']
Tokenized English: ['Resumption', 'of', 'the', 'session']


In [15]:
# helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    for data_sample in data_iter:
        yield token_transform[language](data_sample['translation'][language])

In [16]:
tokens = yield_tokens(train_data, source_language)
# Checking whether tokenization works or not
for token in tokens: 
    print(token)
    break

['Wiederaufnahme', 'der', 'Sitzungsperiode']


#### Text pre-processing

In [17]:
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

In [18]:
vocab_transform = {}   
for ln in [source_language, target_language]:
    # Create torchtext's Vocab object
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_data, ln), min_freq=1, specials=special_symbols)

In [19]:
for ln in [source_language, target_language]:
  vocab_transform[ln].set_default_index(UNK_IDX)

In [20]:
SRC_VOCAB_SIZE = len(vocab_transform[source_language])
TGT_VOCAB_SIZE = len(vocab_transform[target_language])

In [21]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

In [22]:
def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [23]:
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

In [24]:
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]), torch.tensor(token_ids), torch.tensor([EOS_IDX])))

In [25]:
text_transform = {}
for ln in [source_language, target_language]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor

#### Dataloaders

In [26]:
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for element in batch: 
        src_ = element["translation"][source_language]
        tgt_ = element["translation"][target_language]
        src_batch.append(text_transform[source_language](src_))
        tgt_batch.append(text_transform[target_language](tgt_))
    src_batch = nn.utils.rnn.pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = nn.utils.rnn.pad_sequence(tgt_batch, padding_value=PAD_IDX)
    element = {'src': src_batch, 'tgt': tgt_batch}
    return element

train_dataloader = DataLoader(train_data, batch_size=32, collate_fn=collate_fn, shuffle=True)

for i, batch in enumerate(train_dataloader):
    src = batch["src"]
    tgt = batch["tgt"]
    tgt_input = tgt[:-1, :]
    src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
    break

In [27]:
train_dataloader = DataLoader(train_data, batch_size=config["BATCH_SIZE"], collate_fn=collate_fn, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=config["BATCH_SIZE"], collate_fn=collate_fn)
test_dataloader = DataLoader(test_data, batch_size=config["BATCH_SIZE"], collate_fn=collate_fn)

#### Model Architecture

In [28]:
# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout: float, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

In [29]:
# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [30]:
# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self, num_encoder_layers: int, num_decoder_layers: int, emb_size: int, nhead: int, src_vocab_size: int, tgt_vocab_size: int, dim_feedforward: int = 512, dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size, nhead=nhead, num_encoder_layers=num_encoder_layers, num_decoder_layers=num_decoder_layers, dim_feedforward=dim_feedforward, dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(self, src: Tensor, trg: Tensor, src_mask: Tensor, tgt_mask: Tensor, src_padding_mask: Tensor, tgt_padding_mask: Tensor, memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None, src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(self.tgt_tok_emb(tgt)), memory, tgt_mask)

#### WandB Setup

In [26]:
import wandb

In [27]:
wandb.login(relogin=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: ERROR API key must be 40 characters long, yours was 11
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: ERROR API key must be 40 characters long, yours was 11
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an

True

In [28]:
wandb.init(project = "assignment-3", entity = "nlp-assignments", config = config)

wandb: Currently logged in as: prakhar432 (nlp-assignments). Use `wandb login --relogin` to force relogin


#### Training Setup

In [31]:
torch.manual_seed(0)

transformer = Seq2SeqTransformer(config["NUM_ENCODER_LAYERS"], config["NUM_DECODER_LAYERS"], config["EMB_SIZE"], config["NHEAD"], SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, config["FFN_HID_DIM"])

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

c:\Users\prakh\anaconda3\envs\visiongpu\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [30]:
def train_epoch(model, optimizer, dataloader, minibatch):
    model.train()
    losses = 0
    wandb.define_metric('Minibatch_Epoch')
    wandb.define_metric('MiniBatch_Loss')
    for i, batch in enumerate(dataloader):
        src = batch["src"]
        tgt = batch["tgt"]
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
        # logging
        log = {}
        log["Minibatch_Epoch"] = minibatch
        log["MiniBatch_Loss"] = loss.item()
        wandb.log(log)
        minibatch += 1

    return losses / len(list(dataloader)), minibatch

In [31]:
def evaluate(model, dataloader):
    model.eval()
    losses = 0

    for i, batch in enumerate(dataloader):
        src = batch["src"]
        tgt = batch["tgt"]
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()
    return losses / len(list(dataloader))

#### Training Loop

In [32]:
minibatch = 0
for epoch in range(1, config["NUM_EPOCHS"]+1):
    start_time = timer()
    train_loss, minibatch = train_epoch(transformer, optimizer, train_dataloader, minibatch)
    end_time = timer()
    val_loss = evaluate(transformer, val_dataloader)
    wandb.define_metric('Epoch')
    wandb.define_metric('Train_Loss')
    wandb.define_metric('Val_Loss')
    log = {}
    log["Epoch"] = epoch
    log["Train_Loss"] = train_loss
    log["Val_Loss"] = val_loss
    wandb.log(log)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

/home/mehar21541/anaconda3/envs/vis/lib/python3.11/site-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch: 1, Train loss: 5.344, Val loss: 6.414, Epoch time = 3128.391s


In [ ]:
wandb.finish()

Epoch,▁▃▆█
MiniBatch_Loss,█▆▅▄▅▅▄▄▄▄▃▄▄▃▃▃▃▃▂▃▂▃▂▂▂▂▂▂▃▂▁▁▂▂▁▂▁▁▁▁
Minibatch_Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Train_Loss,█▄▂▁
Val_Loss,█▄▂▁
Epoch,4
MiniBatch_Loss,3.78327
Minibatch_Epoch,6251
Train_Loss,3.58966
Val_Loss,5.59713


#### Model Evaluation

In [32]:
# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0)).type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

In [33]:
# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[config["source_language"]](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[config["target_language"]].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [ ]:
# translate German to English
for i in range(5):
    src = train_data[i]["translation"][config["source_language"]]
    print(f"German: {src}")
    print(f"Translated English: {translate(transformer, src)}")
    print(f"Actual English: {train_data[i]['translation'][config['target_language']]}")
    print()

German: Wiederaufnahme der Sitzungsperiode
Translated English:  Vote 
Actual English: Resumption of the session

German: Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie schöne Ferien hatten.


Translated English:  I am pleased to see the session of the European Parliament ' s session of the European Parliament for the European Parliament to ask you to take a good message and hope that you had been able to take
Actual English: I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.

German: Wie Sie feststellen konnten, ist der gefürchtete "Millenium-Bug " nicht eingetreten. Doch sind Bürger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden.
Translated English:  As you have said , the fact that the " " " " " " " " " The citizens are not able to have a few people of our Member States
Actual English: Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.

German:

#### Saving Model

In [ ]:
# save the model
torch.save(transformer.state_dict(), "2A_transformer_3.pth")

In [42]:
# loading the model 
transformer = Seq2SeqTransformer(config["NUM_ENCODER_LAYERS"], config["NUM_DECODER_LAYERS"], config["EMB_SIZE"], config["NHEAD"], SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, config["FFN_HID_DIM"])
transformer.load_state_dict(torch.load("Task_2A/2A_transformer_run_1.pth"))
transformer = transformer.to(DEVICE)

#### Evaluation Metrics Computation

In [43]:
# compute BLEU score using huggingface datasets library
from datasets import load_metric
metric = load_metric("bleu")
# compute actual BLEU score on validation dataset
train_predictions = []
train_references = []
# generate predictions and references for all sentences in the train dataset
for i in tqdm.tqdm(range(len(train_data))):
    src = train_data[i]["translation"][config["source_language"]]
    prediction = translate(transformer, src)
    reference = train_data[i]["translation"][config["target_language"]]
    train_predictions.append(prediction)
    train_references.append([reference])
train_reference = [r[0] for r in train_references]

C:\Users\prakh\AppData\Local\Temp\ipykernel_11632\2238225514.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("bleu")
c:\Users\prakh\anaconda3\envs\visiongpu\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
  0%|          | 0/50000 [00:00<?, ?it/s]c:\Users\prakh\anaconda3\envs\visiongpu\Lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled wi

KeyboardInterrupt: 

In [ ]:
val_predictions = []    
val_references = []
# generate predictions and references for all sentences in the validation dataset
for i in tqdm.tqdm(range(len(val_data))):
    src = val_data[i]["translation"][config["source_language"]]
    prediction = translate(transformer, src)
    reference = val_data[i]["translation"][config["target_language"]]
    val_predictions.append(prediction)
    val_references.append([reference])
val_reference = [r[0] for r in val_references]

In [ ]:
test_predictions = []
test_references = []
# generate predictions and references for all sentences in the test dataset
for i in tqdm.tqdm(range(len(test_data))):
    src = test_data[i]["translation"][config["source_language"]]
    prediction = translate(transformer, src)
    reference = test_data[i]["translation"][config["target_language"]]
    test_predictions.append(prediction)
    test_references.append([reference])
test_reference = [r[0] for r in test_references]

In [ ]:
from evaluate import load
bleu = load("bleu")
train_bleu = bleu(train_predictions, train_references)
val_bleu = bleu(val_predictions, val_references)
test_bleu = bleu(test_predictions, test_references)
print(f"Train BLEU score: {train_bleu}")
print(f"Validation BLEU score: {val_bleu}")
print(f"Test BLEU score: {test_bleu}")

0.028524386585380018


In [ ]:
meteor = load("meteor")
train_meteor = meteor.compute(predictions=train_predictions, references=train_references)
val_meteor = meteor.compute(predictions=val_predictions, references=val_references)
test_meteor = meteor.compute(predictions=test_predictions, references=test_references)
print(f"Train METEOR score: {train_meteor}")
print(f"Validation METEOR score: {val_meteor}")
print(f"Test METEOR score: {test_meteor}")

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/mehar21541/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/mehar21541/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/mehar21541/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


{'meteor': 0.1996686340088016}


In [ ]:
bertScore = load("bertscore")
train_bert = bertScore.compute(predictions=train_predictions, references=train_references, lang="en")
train_precision = np.mean(train_bert["precision"])
train_recall = np.mean(train_bert["recall"])
train_f1 = np.mean(train_bert["f1"])
val_bert = bertScore.compute(predictions=val_predictions, references=val_references, lang="en")
val_precision = np.mean(val_bert["precision"])
val_recall = np.mean(val_bert["recall"])
val_f1 = np.mean(val_bert["f1"])
test_bert = bertScore.compute(predictions=test_predictions, references=test_references, lang="en")
test_precision = np.mean(test_bert["precision"])
test_recall = np.mean(test_bert["recall"])
test_f1 = np.mean(test_bert["f1"])
print(f"Train data: Precision: {train_precision}, Recall: {train_recall}, F1: {train_f1}")
print(f"Validation data: Precision: {val_precision}, Recall: {val_recall}, F1: {val_f1}")
print(f"Test data: Precision: {test_precision}, Recall: {test_recall}, F1: {test_f1}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Precision: 0.8267553925514222, Recall: 0.8387243402004242, F1: 0.8326047211885452


In [ ]:
from nltk.translate.bleu_score import corpus_bleu
def compute_bleu(reference_validation,translations_validation):
    w1 = [1, 0, 0, 0]
    w2 = [0.5, 0.5, 0, 0]
    w3 = [1/3, 1/3, 1/3, 0]
    w4 = [0.25, 0.25, 0.25, 0.25]
    bleu1 = corpus_bleu([[ref] for ref in reference_validation], translations_validation, weights=w1)
    bleu2 = corpus_bleu([[ref] for ref in reference_validation], translations_validation, weights=w2)
    bleu3 = corpus_bleu([[ref] for ref in reference_validation], translations_validation, weights=w3)
    bleu4 = corpus_bleu([[ref] for ref in reference_validation], translations_validation, weights=w4)
    print("BLEU-1 score for validation set:", bleu1)
    print("BLEU-2 score for validation set:", bleu2)
    print("BLEU-3 score for validation set:", bleu3)
    print("BLEU-4 score for validation set:", bleu4)

compute_bleu(train_reference, train_predictions)
compute_bleu(val_reference, val_predictions)
compute_bleu(test_reference, test_predictions)